# 22/02/2021

In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


I am creating two trees. 

One for the momenta components of electron called `e` and one for neutrino called `n`.

Each tree will have 4 branches for `Px`,`Py`,`Pz` and `E`.

In [2]:
e = r.TTree("e", "electronTree")
n = r.TTree("n", "neutrinoTree")

Now I will use the `TLorentzVector` function to generate the data. In this case it is not necessary I suppose, but afterwards, once we need to use `boost` or other things, it will be better if we start with `TLorentzVector` now already.

In [3]:
# Function to generate initial random momentum components for electron
def elMomComponents(mag):
    ''' chooses random theta and phi and then uses it to calculate 
        initial random components of electron momentum
        
        Input:
        mag    magnitude of the momentum
        
        Returns:
        Px
        Py
        Pz
    '''
    theta = np.random.uniform(0,np.pi)
    phi = np.random.uniform(0,2*np.pi)
    
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz

In [4]:
# Setting up 4Vectors for electron and neutrino
e4Vect = r.TLorentzVector() #electron
n4Vect = r.TLorentzVector() #neutrino

In [5]:
# Invariant mass of W boson
massW = 80.3 # Gev/c^2

# energy split equaly in the decay -> massW/2
productE = massW/2

In [6]:
# Setting up the variables, so we can set up branches, where we store them
ePx = np.array([e4Vect.Px()])
ePy = np.array([e4Vect.Py()])
ePz = np.array([e4Vect.Pz()])
eE = np.array([productE])

In [7]:
# Setting up branches to store components of electron momenta 
# (each component is placed separately on each branch)
e.Branch("Px",ePx,"xComponent[1]/D")
e.Branch("Py",ePy,"yComponent[1]/D")
e.Branch("Pz",ePz,"zComponent[1]/D")
e.Branch("E",eE,"energy[1]/D")

In [8]:
# Function to generate 4vector and splits it into different branches
def elMomFill(j, mag, m):
    ''' Uses function elMomComponents to generate random values
        of momenta components with given magnitude of the vector.
        Then it uses TLorentzVector to fill the branch of electron momenta components
        with these randomly generated values.
        
        Input:
        j     number of generated momenta
        mag   magnitude of generated momenta 
        m     energy of the particle
    '''
    
    for i in range(j):
        Px, Py, Pz = elMomComponents(mag)
        e4Vect.SetPxPyPzE(Px, Py, Pz, m)

        # Mass won't change, so we do not need it here
        ePx[0] = e4Vect.Px()
        ePy[0] = e4Vect.Py()
        ePz[0] = e4Vect.Pz()
        eE[0] = e4Vect.E()

        e.Fill()

In [9]:
elMomFill(3,1,productE)

In [10]:
e.Scan("Px:Py:Pz:E")

3

************************************************************
*    Row   *        Px *        Py *        Pz *         E *
************************************************************
*        0 * 0.9545641 * 0.1866936 * -0.232277 *     40.15 *
*        1 * -0.874949 * 0.2649212 * -0.405313 *     40.15 *
*        2 * 0.5619539 * -0.733944 * 0.3814876 *     40.15 *
************************************************************


In [11]:
print("Nice!")

Nice!


Now we will calculate the neutrino momenta.

If I understand it correctly:
$$ 80.3^2 = (E_e + E_{\nu})^2 - \lvert \sum \textbf{p} \rvert ^2  $$

$$ 80.3^2 = 80.3^2 - (p_z + p_{z\nu})^2 $$

$$ p_{z\nu}^2 + 2p_zp_{z\nu} + p_z^2 = 0 $$

$$ p_{z\nu} = - p_z $$

So in the stationary W case, electron and neutrino will start moving back to back, so the spatial components will be the same but will have opposite sign?

In that case we can use this to set up another tree with neutrino components:

In [12]:
# Setting up the variables, so we can set up branches, where we store them
nPx = np.array([n4Vect.Px()])
nPy = np.array([n4Vect.Py()])
nPz = np.array([n4Vect.Pz()])
nE = np.array([productE])

In [13]:
# Setting up branches to store components of neutrino momenta 
# (each component is placed separately on each branch)
n.Branch("Px",nPx,"xComponent[1]/D")
n.Branch("Py",nPy,"yComponent[1]/D")
n.Branch("Pz",nPz,"zComponent[1]/D")
n.Branch("E",nE,"energy[1]/D")

In [14]:
# Finding number of entries in the electron tree
e.GetEntries()

3

In [15]:
def neuMomGenerator():
    ''' Generates neutrino momenta components for back to back case
        and fills them into neutrino tree. 
        Same result as elMomGenerator().
    '''
    for entry in e:
        x = -entry.Px
        y = -entry.Py
        z = -entry.Pz
        varE = entry.E

        n4Vect.SetPxPyPzE(x,y,z,varE) #Changing TLorentzVector for neutrino

        nPx[0] = n4Vect.Px()
        nPy[0] = n4Vect.Py()
        nPz[0] = n4Vect.Pz()
        nE[0] = n4Vect.E()

        n.Fill()

In [16]:
# Calling the function
neuMomGenerator()

Now we can look at how the tree looks like.

In [17]:
e.Scan("Px:Py:Pz:E")
n.Scan("Px:Py:Pz:E")

3

************************************************************
*    Row   *        Px *        Py *        Pz *         E *
************************************************************
*        0 * 0.9545641 * 0.1866936 * -0.232277 *     40.15 *
*        1 * -0.874949 * 0.2649212 * -0.405313 *     40.15 *
*        2 * 0.5619539 * -0.733944 * 0.3814876 *     40.15 *
************************************************************
************************************************************
*    Row   *        Px *        Py *        Pz *         E *
************************************************************
*        0 * -0.954564 * -0.186693 * 0.2322772 *     40.15 *
*        1 * 0.8749499 * -0.264921 * 0.4053138 *     40.15 *
*        2 * -0.561953 * 0.7339448 * -0.381487 *     40.15 *
************************************************************


In [18]:
# Now we create the file and put both trees into it
file = r.TFile("stationaryW_case.root","recreate")
file.Write()
file.Close()

This looks like it could be right and we can check with these values if we decide to read the `stationaryW_case.root` file. Now let's figure out how to read this.